In [1]:
from numpy import *
import os
from pylab import *
import numpy as np
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from numpy import random



import tensorflow as tf

#from caffe_classes import class_names

train_x = zeros((1, 227,227,3)).astype(float32)
train_y = zeros((1, 1000))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]

In [2]:
im1 = (imread("laska.png")[:,:,:3]).astype(float32)
im1 = im1 - mean(im1)
im1[:, :, 0], im1[:, :, 2] = im1[:, :, 2], im1[:, :, 0]

im2 = (imread("poodle.png")[:,:,:3]).astype(float32)
im2[:, :, 0], im2[:, :, 2] = im2[:, :, 2], im2[:, :, 0]

/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.
/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """


In [3]:
net_data = load(open("bvlc_alexnet.npy", "rb"), encoding="latin1").item()

In [4]:
def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        #tf.split(value, num_or_size_splits, axis) -- new version
        ## tf.split(axis, num_or_size_splits, value) -- old version
        
        #input_groups = tf.split(3, group, input)
        #kernel_groups = tf.split(3, group, kernel)
        
        input_groups = tf.split(input, group, 3)
        kernel_groups = tf.split(kernel, group, 3)
        
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat( output_groups, 3)
    return  tf.reshape(tf.nn.bias_add(conv, biases), [-1]+conv.get_shape().as_list()[1:])



x = tf.placeholder(tf.float32, (None,) + xdim)

In [5]:
k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
conv1W = tf.Variable(net_data["conv1"][0])
conv1b = tf.Variable(net_data["conv1"][1])
conv1_in = conv(x, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
conv1 = tf.nn.relu(conv1_in)

#lrn1
#lrn(2, 2e-05, 0.75, name='norm1')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn1 = tf.nn.local_response_normalization(conv1,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool1
#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)


#conv2
#conv(5, 5, 256, 1, 1, group=2, name='conv2')
k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
conv2W = tf.Variable(net_data["conv2"][0])
conv2b = tf.Variable(net_data["conv2"][1])
conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv2 = tf.nn.relu(conv2_in)


#lrn2
#lrn(2, 2e-05, 0.75, name='norm2')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn2 = tf.nn.local_response_normalization(conv2,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool2
#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#conv3
#conv(3, 3, 384, 1, 1, name='conv3')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
conv3W = tf.Variable(net_data["conv3"][0])
conv3b = tf.Variable(net_data["conv3"][1])
conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv3 = tf.nn.relu(conv3_in)

#conv4
#conv(3, 3, 384, 1, 1, group=2, name='conv4')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
conv4W = tf.Variable(net_data["conv4"][0])
conv4b = tf.Variable(net_data["conv4"][1])
conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv4 = tf.nn.relu(conv4_in)


#conv5
#conv(3, 3, 256, 1, 1, group=2, name='conv5')
k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
conv5W = tf.Variable(net_data["conv5"][0])
conv5b = tf.Variable(net_data["conv5"][1])
conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv5 = tf.nn.relu(conv5_in)

#maxpool5
#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#fc6
#fc(4096, name='fc6')
fc6W = tf.Variable(net_data["fc6"][0])
fc6b = tf.Variable(net_data["fc6"][1])
fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(prod(maxpool5.get_shape()[1:]))]), fc6W, fc6b)

#fc7
#fc(4096, name='fc7')
fc7W = tf.Variable(net_data["fc7"][0])
fc7b = tf.Variable(net_data["fc7"][1])
fc7 = tf.nn.relu_layer(fc6, fc7W, fc7b)

fc7_xw_plus_b = nn_ops.bias_add(math_ops.matmul(fc6, fc7W), fc7b)


fc7_after_relu = nn_ops.relu(fc7_xw_plus_b)


#fc8
#fc(1000, relu=False, name='fc8')
fc8W = tf.Variable(net_data["fc8"][0])
fc8b = tf.Variable(net_data["fc8"][1])
fc8 = tf.nn.xw_plus_b(fc7, fc8W, fc8b)


#prob
#softmax(name='prob'))
prob = tf.nn.softmax(fc8)

#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

t = time.time()
#output = sess.run(prob, feed_dict = {x:[im1,im2]})
################################################################################

#fc7_xw_plus_b_output = sess.run(fc7_xw_plus_b, feed_dict = {x:[im1,im2]})

#Output:
'''
for input_im_ind in range(output.shape[0]):
    inds = argsort(output)[input_im_ind,:]
    print "Image", input_im_ind
    for i in range(5):
        print class_names[inds[-1-i]], output[input_im_ind, inds[-1-i]]
'''
#print 'two-step ReLu:',fc7_xw_plus_b_output[1,:]


'\nfor input_im_ind in range(output.shape[0]):\n    inds = argsort(output)[input_im_ind,:]\n    print "Image", input_im_ind\n    for i in range(5):\n        print class_names[inds[-1-i]], output[input_im_ind, inds[-1-i]]\n'

In [6]:
ls

AlexNet.ipynb                          poodle.png
Untitled.ipynb                         ref_im_fc7.npy
Visual.ipynb                           task_test/
alexnet_imagenet_distribution/         task_train/
bvlc_alexnet.npy                       test_image.jpg
caffe_classes.py                       test_part_A.npy
cal_comp_hist.ipynb                    test_part_B.npy
crop.ipynb                             test_set.npy
cute.jpg                               tgt_im_fc7.npy
gt_test.csv                            train test AB dividing.ipynb
gt_train.csv                           train2017/
half half _ benchmark II-test.ipynb    train_part_A.npy
half half _ benchmark II-train.ipynb   train_part_B.npy
half half _ benchmark II.ipynb         train_set.npy
half half _ benchmark.ipynb            train_task_choices_minus_1.npy
intelligent searching Li's part.ipynb  train_task_ref.npy
laska.png                              train_task_tgt.npy


In [7]:
ref_im_list = []
tgt_im_list = []

num_choices = 10
cnt = 0

ref_im_fc7 = []
tgt_im_fc7 = []

for sample in os.listdir('task_train/'):
    if cnt == 2000:
        break
        
    if sample != '.DS_Store' and sample != 'README.md':
        ref_im_name = 'task_train/' +sample+ '/reference.jpg'
        ref_im = imread(ref_im_name)
        ref_im = imresize(ref_im,(227,227,3))
        ref_im = (ref_im[:,:,:3]).astype(float32)
        ref_im = ref_im - mean(ref_im)
        ref_im[:, :, 0], ref_im[:, :, 2] = ref_im[:, :, 2], ref_im[:, :, 0]
        ref_im_list.append(ref_im)
    
        tmp_tgt_im_list = []
        tmp_tgt_im_fc7  = []
        
        for i in range(num_choices):
            tgt_im_name = 'task_train/' +sample+ '/choice_' + str(i) + '.jpg'
            tgt_im = imread(tgt_im_name)
            tgt_im = imresize(tgt_im, (227,227,3))
            tgt_im = (tgt_im[:,:,:3]).astype(float32)
            tgt_im = tgt_im - mean(tgt_im)
            tgt_im[:, :, 0], tgt_im[:, :, 2] = tgt_im[:, :, 2], tgt_im[:, :, 0]
            tmp_tgt_im_list.append(tgt_im)
        
        tmp_tgt_im_fc7 = sess.run(fc7_xw_plus_b, feed_dict = {x:tmp_tgt_im_list})
        
        tgt_im_list.append(tmp_tgt_im_list)
        tgt_im_fc7.append(tmp_tgt_im_fc7)
        
    
    cnt += 1

ref_im_fc7 = sess.run(fc7_xw_plus_b, feed_dict = {x:ref_im_list})

ref_im_list = np.array(ref_im_list)
tgt_im_list = np.array(tgt_im_list)
ref_im_fc7 = np.array(ref_im_fc7)
tgt_im_fc7 = np.array(tgt_im_fc7)

print(ref_im_list.shape)
print(tgt_im_list.shape) 
print(ref_im_fc7.shape)
print(tgt_im_fc7.shape)
    

/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(1999, 227, 227, 3)
(1999, 10, 227, 227, 3)
(1999, 4096)
(1999, 10, 4096)


In [8]:
np.save('ref_im_fc7.npy', ref_im_fc7)
np.save('tgt_im_fc7.npy', tgt_im_fc7)